# NLP for Fake News detection
Applying Neural Language Processing to try and identify fake news from 'true' news. 

Dataset available at: https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset 


### Data exploration

In [0]:
# importing necessary libs
import pandas as pd
import numpy as np

In [5]:
# uploading csv files
from google.colab import files
uploaded = files.upload()

Saving Fake.csv to Fake.csv
Saving True.csv to True.csv


In [0]:
# Dataset is now stored in a Pandas Dataframe
import io
fake_news = pd.read_csv(io.BytesIO(uploaded['Fake.csv']))
true_news = pd.read_csv(io.BytesIO(uploaded['True.csv']))

In [7]:
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [8]:
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [9]:
# number of columns and rows of each dataframe
fake_news.shape, true_news.shape

((23481, 4), (21417, 4))

In [10]:
fake_news.describe().T

,count,unique,top,freq
title,23481,17903,MEDIA IGNORES Time That Bill Clinton FIRED His...,6
text,23481,17455,,626
subject,23481,6,News,9050
date,23481,1681,"May 10, 2017",46


In [11]:
true_news.describe().T

,count,unique,top,freq
title,21417,20826,Factbox: Trump fills top jobs for his administ...,14
text,21417,21192,(Reuters) - Highlights for U.S. President Dona...,8
subject,21417,2,politicsNews,11272
date,21417,716,"December 20, 2017",182


In [0]:
# adding feature that will be target variable True/Fake
fake_news['True/Fake'] = 'True'
true_news['True/Fake'] = 'Fake'

In [0]:
# combining the dataframes into a single one using concat method
total_news = pd.concat([true_news, fake_news])
total_news['Article'] = total_news['title'] + total_news['text'] # merging columns title and text

In [14]:
# shuffling dataset 100% in order to mix True and Fake
total_news.sample(frac=1)

,title,text,subject,date,True/Fake,Article
695,GOP Congress Just Delivered Trump The Biggest...,If there s one thing Donald Trump fears the mo...,News,"August 1, 2017",True,GOP Congress Just Delivered Trump The Biggest...
13023,JOHN KERRY’S STATE DEPT Reportedly Funneled Ov...,So much for the most transparent administrati...,politics,"Sep 13, 2016",True,JOHN KERRY’S STATE DEPT Reportedly Funneled Ov...
8883,Trump defends 'star' tweet; Clinton says it's ...,NEW YORK (Reuters) - Republican presidential c...,politicsNews,"July 4, 2016",Fake,Trump defends 'star' tweet; Clinton says it's ...
10187,BREAKING NEWS: GENERAL MATTIS Issues Fiery War...,President Trump put North Korea s brutal dicta...,politics,"Aug 9, 2017",True,BREAKING NEWS: GENERAL MATTIS Issues Fiery War...
21913,LISTEN TO THIS FORMER DOJ WHISTLEBLOWER,THE CONGRESS IS THERE TO DO WHAT S BEST FOR TH...,left-news,"Apr 5, 2015",True,LISTEN TO THIS FORMER DOJ WHISTLEBLOWERTHE CON...
...,...,...,...,...,...,...
10159,"Trump questions NATO, Asia nuclear weapons ahe...",WASHINGTON (Reuters) - U.S. Republican preside...,politicsNews,"March 27, 2016",Fake,"Trump questions NATO, Asia nuclear weapons ahe..."
19207,WHAT? DEMOCRAT CONGRESSWOMAN Calls Violent Rio...,Here s just one video from last night that wil...,left-news,"Feb 2, 2017",True,WHAT? DEMOCRAT CONGRESSWOMAN Calls Violent Rio...
7242,U.S. panel urges probe on whether China weaken...,WASHINGTON (Reuters) - A U.S. advisory commiss...,politicsNews,"November 16, 2016",Fake,U.S. panel urges probe on whether China weaken...
1810,Trump calls for a tax reform 'speed-up' in lig...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"September 10, 2017",Fake,Trump calls for a tax reform 'speed-up' in lig...


## Basic Data Cleaning

In [18]:
# downloading stopword from nltk lib
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
# importing libs
from nltk.corpus import stopwords
import string

In [0]:
# build function to process text
def process_text(s):

    # Check each string for punctuation
    no_punc = [char for char in s if char not in string.punctuation]

    # Join the characters again 
    no_punc = ''.join(no_punc)
    
    # Convert string to lowercase and remove stopwords
    clean_string = [word for word in no_punc.split() if word.lower() not in stopwords.words('english')]
    return clean_string

In [0]:
# Tokenize article so words can be represented by numbers
# this process might take a while: avg 30 mim
total_news['Clean Text'] = total_news['Article'].apply(process_text)

In [22]:
total_news.sample(5)

,title,text,subject,date,True/Fake,Article,Clean Text
8494,Bernie Sanders Just Opened Up About Something...,In a recent candid interview with the Washingt...,News,"January 27, 2016",True,Bernie Sanders Just Opened Up About Something...,"[Bernie, Sanders, Opened, Something, Make, Con..."
14934,France pays tribute to Paris dead two years on,"PARIS (Reuters) - France, still on alert again...",worldnews,"November 13, 2017",Fake,France pays tribute to Paris dead two years on...,"[France, pays, tribute, Paris, dead, two, year..."
15531,TRUMP RESPONSE TO LEFTIST THREATS: “Get Off My...,It s actually quite refreshing to see someone ...,politics,"Jun 27, 2015",True,TRUMP RESPONSE TO LEFTIST THREATS: “Get Off My...,"[TRUMP, RESPONSE, LEFTIST, THREATS, “Get, Lawn..."
6918,China newspapers say call with Taiwan's Tsai s...,SHANGHAI (Reuters) - Chinese state media on Mo...,politicsNews,"December 5, 2016",Fake,China newspapers say call with Taiwan's Tsai s...,"[China, newspapers, say, call, Taiwans, Tsai, ..."
2349,Dem AGs Respond To Trump Rolling Back Clean W...,Amateur president Donald Trump s hostility tow...,News,"March 1, 2017",True,Dem AGs Respond To Trump Rolling Back Clean W...,"[Dem, AGs, Respond, Trump, Rolling, Back, Clea..."


## Vectorization
Convert the collection of text documents to a matrix of token counts.




In [0]:
# importing libs
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
# Bag-of-Words (bow) transform the entire DataFrame of text
# transforming tokens into useful features
bow_transformer = CountVectorizer(analyzer=process_text).fit(total_news['Clean Text'])

In [25]:
# sum of words after vectorizing
print(f'Total vocabulary words: {len(bow_transformer.vocabulary_)}') 

Total vocabulary words: 39099


In [0]:
# assigning the matrix of tokens to news_bow variable
news_bow = bow_transformer.transform(total_news['Clean Text'])

In [27]:
# printing out the shape of our Matrix
print(f'Shape of Sparse Matrix: {news_bow.shape}')
print(f'Amount of Non-Zero occurences: {news_bow.nnz}')

Shape of Sparse Matrix: (44898, 39099)
Amount of Non-Zero occurences: 44898


In [28]:
# checking out sparsity
# sparsity of 0 means its fully dense
sparsity = (100.0 * news_bow.nnz / (news_bow.shape[0] * news_bow.shape[1]))
print(f'sparsity: {round(sparsity)}')

sparsity: 0


In [0]:
# extracting text with Transformer
from sklearn.feature_extraction.text import TfidfTransformer

In [31]:
# checking news_bow shape
tfidf_transformer = TfidfTransformer().fit(news_bow)
news_tfidf = tfidf_transformer.transform(news_bow)
print(news_tfidf.shape)

(44898, 39099)


### Build and train the model

In [0]:
# importing algorithm to build the model
from sklearn.naive_bayes import MultinomialNB

In [0]:
fakenews_detect_model = MultinomialNB().fit(news_tfidf, total_news['True/Fake'])

In [34]:
# Model Evaluation
predictions = fakenews_detect_model.predict(news_tfidf)
print(predictions)

['Fake' 'Fake' 'Fake' ... 'True' 'True' 'True']


In [35]:
# importing metrics to check predictions
from sklearn.metrics import classification_report
print (classification_report(total_news['True/Fake'], predictions))

              precision    recall  f1-score   support

        Fake       1.00      1.00      1.00     21417
        True       1.00      1.00      1.00     23481

    accuracy                           1.00     44898
   macro avg       1.00      1.00      1.00     44898
weighted avg       1.00      1.00      1.00     44898



### Building pipeline

In [0]:
# importing train_test_split to split the data and train the model
from sklearn.model_selection import train_test_split

news_train, news_test, text_train, text_test = train_test_split(total_news['Article'], total_news['True/Fake'], test_size=0.3)

In [37]:
# building pipeline
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=process_text)),  
    ('tfidf', TfidfTransformer()),  
    ('classifier', MultinomialNB()),  
])
pipeline.fit(news_train,text_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function process_text at 0x7fd734572c80>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [0]:
# predicting news on test dataset
prediction = pipeline.predict(news_test)

In [39]:
# printing out classification report
print(classification_report(prediction,text_test))

              precision    recall  f1-score   support

        Fake       0.98      0.95      0.97      6491
        True       0.96      0.98      0.97      6979

    accuracy                           0.97     13470
   macro avg       0.97      0.97      0.97     13470
weighted avg       0.97      0.97      0.97     13470



### Model evaluation
Multinomial Naive Bayes Algorithm performs well, sucessfully predicting both fake and true news, resulting in a total accuracy of 0.97.
Further anaylsis are recommended to verify if it's a case of overfitting. 


